L'objectif de ce notebook est de fournir une analyse descriptive du fichier "Classement thématique des sujets de journeaux télévisés", qui fera l'objet d'un tableau de bord.

Vincent Lucas, 01/2025

# Imports

In [ ]:
!pip install streamlit
!pip install gradio
!pip install squarify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from google.colab import drive
import matplotlib.pyplot as plt
import os
import pandas as pd
import gradio as gr
import io
import seaborn as sns
import squarify
import plotly.express as px


# Connexion au drive

In [ ]:
drive.mount("/content/drive", force_remount=True)
chemin_donnees = "/content/drive/MyDrive/PIP2025/Donnees"

MessageError: Error: credential propagation was unsuccessful

In [ ]:
def afficher_arborescence(chemin, prefix=""):
    try:
        elements = os.listdir(chemin)
    except PermissionError:
        print(prefix + "└── [Accès refusé] " + os.path.basename(chemin))
        return

    elements.sort()

    for i, element in enumerate(elements):
        chemin_element = os.path.join(chemin, element)

        if i == len(elements) - 1:
            connector = "└── "
            extension = "    "
        else:
            connector = "├── "
            extension = "│   "

        print(prefix + connector + element)

        if os.path.isdir(chemin_element):
            afficher_arborescence(chemin_element, prefix + extension)

In [ ]:
afficher_arborescence(chemin_donnees)

In [ ]:
dossiers_utiles = ["Audience de la télévision", "Classement thématique des sujets de journeaux télévisés", "Programmes audiovisuels (offre, consommation, coût de grille, investissements)"]

# Stats descriptives

In [ ]:
def stats_descriptives(df, nom_fichier):
    """
    Affiche quelques statistiques descriptives de base
    pour un DataFrame pandas.
    """
    print(f"\nFichier : {nom_fichier}")
    print(f"Nombre de lignes  : {df.shape[0]}")
    print(f"Nombre de colonnes: {df.shape[1]}")
    print("Nom des colonnes  : ", list(df.columns))
    print("\nDescription statistique :")
    # .describe() donne des stats sur les colonnes numériques
    print(df.describe(include='all'))  # pour inclure aussi stats sur colonnes object

In [ ]:
data_frames = {}

for dossier in dossiers_utiles:
    chemin_dossier = os.path.join(chemin_donnees, dossier)

    if not os.path.isdir(chemin_dossier):
        print(f"\nLe dossier {chemin_dossier} n'existe pas ou n'est pas accessible.")
        continue

    print(f"\n=== Parcours du dossier : {dossier} ===")

    # On parcourt tous les fichiers du dossier
    for fichier in os.listdir(chemin_dossier):
        chemin_fichier = os.path.join(chemin_dossier, fichier)

        # Vérifie qu'il s'agit bien d'un fichier et non d'un sous-dossier
        if os.path.isfile(chemin_fichier):
            extension = os.path.splitext(fichier)[1].lower()  # ex: .csv, .xlsx, etc.

            # Gestion des fichiers CSV
            if extension == '.csv':
                try:
                    # Premier essai : encodage 'utf-8'
                    df = pd.read_csv(chemin_fichier, encoding='utf-8')
                except UnicodeDecodeError:
                    # Si problème, on tente latin-1
                    print(f"Problème d'encodage en utf-8 pour le fichier {fichier}. On tente latin-1.")
                    try:
                        df = pd.read_csv(chemin_fichier, encoding='latin-1')
                    except Exception as e:
                        print(f"Impossible de lire le fichier {fichier} : {e}")
                        continue  # On passe au fichier suivant
                except Exception as e:
                    print(f"Erreur lors de la lecture du fichier {fichier} : {e}")
                    continue  # On passe au fichier suivant

                # On stocke le DataFrame dans le dictionnaire
                data_frames[fichier] = df

                # Affichage des stats
                stats_descriptives(df, fichier)

            # Gestion des fichiers Excel
            elif extension in ['.xlsx', '.xls']:
                try:
                    df = pd.read_excel(chemin_fichier)
                    # On stocke le DataFrame
                    data_frames[fichier] = df
                    # Affichage des stats
                    stats_descriptives(df, fichier)
                except Exception as e:
                    print(f"Erreur lors de la lecture du fichier {fichier} : {e}")
                    continue

# Classement thématique des sujets de journeaux télévisés

In [ ]:
path = os.path.join(chemin_donnees, "Classement thématique des sujets de journeaux télévisés", "ina-barometre-jt-tv-donnees-quotidiennes-2000-2020-nbre-sujets-durees-202410.csv")
df = pd.read_csv(
    path,
    encoding='latin-1',
    sep=';',
    header=None,
    names=["Date", "Chaine", "Vide", "Theme", "NbEmissions", "Temps"]
)
df.dropna(axis=1, how='all', inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Annee'] = df['Date'].dt.year.astype(int)
df

In [ ]:
df_par_annee = df.groupby(df['Date'].dt.year)['NbEmissions'].sum()

# 3) Optionnel : transformer la série en DataFrame pour plus de souplesse
df_par_annee = df_par_annee.reset_index()
df_par_annee.columns = ['Annee', 'NbEmissionsTotal']

# 4) Affichage du graphique en barres
plt.figure(figsize=(10, 6))  # Ajustez la taille selon vos préférences
plt.bar(df_par_annee['Annee'], df_par_annee['NbEmissionsTotal'], color='skyblue')
plt.xlabel('Année')
plt.ylabel("Nombre d'émissions")
plt.title("Nombre total d'émissions par année")
plt.xticks(rotation=45)  # Rotation de l’axe des X pour mieux lire les années
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_par_annee['Annee'], df_par_annee['NbEmissionsTotal'], marker='o', linestyle='-')
plt.xlabel('Année')
plt.ylabel("Nombre d'émissions")
plt.title("Nombre total d'émissions par année")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df_grouped = df.groupby([df['Date'].dt.year, 'Theme'])['Temps'].sum().reset_index()
df_grouped.rename(columns={'Date': 'Annee'}, inplace=True)

df_pivot = df_grouped.pivot(index='Annee', columns='Theme', values='Temps')

plt.figure(figsize=(12, 6))
df_pivot.plot(marker='o', ax=plt.gca())

plt.title("Temps total d'émissions par année et par thème")
plt.xlabel('Année')
plt.ylabel("Temps d'émissions")

plt.xticks(rotation=45)

plt.tight_layout()

plt.legend(title='Thème', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
df_grouped = df.groupby(['Annee', 'Theme'])['Temps'].sum().reset_index()

# 3) Passer de secondes à heures
df_grouped['Temps'] = df_grouped['Temps'] / 3600  # 3600 sec = 1 heure

# 4) Construire un pivot pour avoir :
#    - index : Annee
#    - colonnes : chaque Theme
#    - valeurs : temps (en heures)
df_pivot = df_grouped.pivot(index='Annee', columns='Theme', values='Temps')

# 5) Tracer un diagramme en lignes pour tous les thèmes
plt.figure(figsize=(12, 6))

df_pivot.plot(marker='o', ax=plt.gca())  # on utilise gca() pour tracer sur l'axe courant

plt.title("Temps total d'émission (en heures) par année et par thème")
plt.xlabel('Année')
plt.ylabel("Temps d'émission (heures)")

# Forcer l'axe X à afficher les années comme des entiers
plt.xticks(df_pivot.index, df_pivot.index.astype(int), rotation=45)

plt.tight_layout()

plt.legend(title='Thème', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
df_grouped = df.groupby(['Annee', 'Chaine'])['Temps'].sum().reset_index()

# 4) Convertir le temps de secondes en heures
df_grouped['Temps'] = df_grouped['Temps'] / 3600  # 3600 sec = 1 heure

# 5) Créer un pivot table :
#    - index : Année
#    - colonnes : Chaîne
#    - valeurs : Temps (en heures)
df_pivot = df_grouped.pivot(index='Annee', columns='Chaine', values='Temps')

# 6) Tracer un diagramme en lignes pour toutes les chaînes
plt.figure(figsize=(12, 6))
df_pivot.plot(marker='o', ax=plt.gca())  # on utilise gca() pour tracer sur l'axe courant

plt.title("Temps total d'émission (en heures) par année et par chaîne")
plt.xlabel('Année')
plt.ylabel("Temps d'émission (heures)")

# Forcer l'axe X à afficher les années comme des entiers
plt.xticks(df_pivot.index, df_pivot.index.astype(int), rotation=45)

# Ajuster l'espacement
plt.tight_layout()

# Afficher la légende (chaque chaîne aura sa propre courbe)
plt.legend(title='Chaîne', bbox_to_anchor=(1.05, 1), loc='upper left')

# Afficher le graphique
plt.show()

# Dashboard

In [ ]:
liste_chaines = df["Chaine"].unique().tolist()
liste_themes = df["Theme"].unique().tolist()

## Graphe en lignes

In [ ]:
liste_chaines = df["Chaine"].unique().tolist()
liste_themes = df["Theme"].unique().tolist()
def generer_graphique(chaine_selection, theme_selection):

    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]
    grouped = df_filtered.groupby(["Annee", "Chaine"])["Temps"].sum().reset_index()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    if grouped.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    df_pivot = grouped.pivot(index="Annee", columns="Chaine", values="Temps_heures")

    fig, ax = plt.subplots(figsize=(6, 4))
    df_pivot.plot(marker="o", ax=ax)
    ax.set_title("Temps total d'émission (heures) par année et par chaîne")
    ax.set_xlabel("Année")
    ax.set_ylabel("Temps d'émission (heures)")
    ax.legend(title="Chaîne", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()

    # On renvoie directement la figure Matplotlib
    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )
    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )
    btn = gr.Button("Générer le graphique")

    # gr.Plot() pour un graphique Matplotlib
    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_graphique,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch()


## Histogramme

In [ ]:
def generer_histogramme(chaine_selection, theme_selection):
    """
    1) Filtre le DataFrame sur la base des chaînes et thèmes sélectionnés
    2) Calcule la somme de 'Temps' par année et par chaîne
    3) Convertit en heures
    4) Fait un histogramme Matplotlib
    5) Renvoye le graphique Matplotlib et un aperçu du DataFrame filtré
    """
    # -- 1) Filtrage --
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]

    # -- 2) Groupement par Année et Chaine, somme du Temps --
    grouped = df_filtered.groupby(["Annee", "Chaine"])["Temps"].sum().reset_index()

    # -- 3) Convertir en heures --
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    # -- 4) Pivot table pour tracer l'histogramme (une barre par chaîne pour chaque année) --
    if grouped.empty:
        # S'il n'y a pas de données filtrées, on renvoie juste un message et un DataFrame vide
        return "Aucune donnée après filtrage", df_filtered.head(10)

    df_pivot = grouped.pivot(index="Annee", columns="Chaine", values="Temps_heures").fillna(0)

    # Création de l'histogramme
    fig, ax = plt.subplots(figsize=(10, 6))

    # Définir la largeur des barres et les positions
    bar_width = 0.15
    years = df_pivot.index.tolist()
    x = range(len(years))

    # Nombre de chaînes
    n_chaines = len(df_pivot.columns)

    # Palette de couleurs
    colors = plt.colormaps['tab10'].colors  # Mise à jour pour éviter la dépréciation

    # Tracer chaque chaîne avec un décalage
    for i, chaine in enumerate(df_pivot.columns):
        temps = df_pivot[chaine].tolist()
        ax.bar([p + bar_width*i for p in x], temps, width=bar_width, label=chaine, color=colors[i % len(colors)])

    # Configuration des axes
    ax.set_title("Temps total d'émission (heures) par année et par chaîne")
    ax.set_xlabel("Année")
    ax.set_ylabel("Temps d'émission (heures)")
    ax.set_xticks([p + bar_width*(n_chaines/2) for p in x])
    ax.set_xticklabels(years, rotation=45)
    ax.legend(title="Chaîne")
    plt.tight_layout()

    # -- 5) Retour : on renvoie la figure Matplotlib et le DataFrame filtré --
    return fig, df_filtered.head(10)

# Création de l'interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Dashboard d'exemple : Temps d'émission par année (Histogramme Gradio)")

    # Composant "CheckboxGroup" pour les chaînes
    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines  # sélection par défaut = toutes
    )

    # Composant "CheckboxGroup" pour les thèmes
    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes  # sélection par défaut = tous
    )

    # Bouton pour déclencher l'opération
    btn = gr.Button("Générer l'histogramme")

    # Sorties : plot + dataframe
    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    # Lorsque le bouton est cliqué, on appelle la fonction generer_histogramme
    btn.click(
        fn=generer_histogramme,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

# Lancement de l'application Gradio
demo.launch(debug=True)

## Diagramme camembert

In [ ]:
def generer_camembert(chaine_selection, theme_selection):
    """
    1) Filtre le DataFrame sur la base des chaînes et thèmes sélectionnés
    2) Calcule la somme de 'Temps' par chaîne
    3) Convertit en heures
    4) Fait un diagramme camembert Matplotlib
    5) Renvoye le graphique Matplotlib et un aperçu du DataFrame filtré
    """
    # -- 1) Filtrage --
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]

    # -- 2) Groupement par Chaine, somme du Temps --
    grouped = df_filtered.groupby("Chaine")["Temps"].sum().reset_index()

    # -- 3) Convertir en heures --
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    # -- 4) Création du diagramme camembert --
    if grouped.empty:
        # S'il n'y a pas de données filtrées, on renvoie juste un message et un DataFrame vide
        return "Aucune donnée après filtrage", df_filtered.head(10)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(
        grouped["Temps_heures"],
        labels=grouped["Chaine"],
        autopct='%1.1f%%',
        startangle=140,
        colors=plt.colormaps['tab10'].colors[:len(grouped)]
    )
    ax.set_title("Répartition du temps d'émission (heures) par chaîne")
    ax.axis('equal')  # Assure que le camembert est circulaire

    plt.tight_layout()

    # -- 5) Retour : on renvoie la figure Matplotlib et le DataFrame filtré --
    return fig, df_filtered.head(10)

# Création de l'interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Dashboard d'exemple : Répartition du Temps d'Émission par Chaîne (Camembert Gradio)")

    # Composant "CheckboxGroup" pour les chaînes
    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines  # sélection par défaut = toutes
    )

    # Composant "CheckboxGroup" pour les thèmes
    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes  # sélection par défaut = tous
    )

    # Bouton pour déclencher l'opération
    btn = gr.Button("Générer le camembert")

    # Sorties : plot + dataframe
    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    # Lorsque le bouton est cliqué, on appelle la fonction generer_camembert
    btn.click(
        fn=generer_camembert,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

# Lancement de l'application Gradio
demo.launch(debug=True)

## Diagramme barres empilées

In [ ]:
def generer_barres_empilees(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]
    grouped = df_filtered.groupby(["Annee", "Chaine"])["Temps"].sum().reset_index()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    if grouped.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    df_pivot = grouped.pivot(index="Annee", columns="Chaine", values="Temps_heures").fillna(0)

    fig, ax = plt.subplots(figsize=(10, 6))
    df_pivot.plot(kind='bar', stacked=True, ax=ax, colormap='tab10')
    ax.set_title("Temps total d'émission (heures) par année et par chaîne")
    ax.set_xlabel("Année")
    ax.set_ylabel("Temps d'émission (heures)")
    ax.legend(title="Chaîne", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()

    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    gr.Markdown("## Diagramme en Barres Empilées : Temps d'émission par année et par chaîne")

    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )

    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )

    btn = gr.Button("Générer les barres empilées")

    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_barres_empilees,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch(debug=True)

## Heatmap

In [ ]:
def generer_heatmap(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]
    grouped = df_filtered.groupby(["Annee", "Chaine"])["Temps"].sum().reset_index()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    if grouped.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    df_pivot = grouped.pivot(index="Annee", columns="Chaine", values="Temps_heures").fillna(0)

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(df_pivot, annot=True, fmt=".1f", cmap="YlGnBu", ax=ax)
    ax.set_title("Heatmap du Temps d'émission (heures) par année et par chaîne")
    ax.set_xlabel("Chaîne")
    ax.set_ylabel("Année")
    plt.tight_layout()

    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    gr.Markdown("## Heatmap : Temps d'émission par année et par chaîne")

    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )

    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )

    btn = gr.Button("Générer la heatmap")

    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_heatmap,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch(debug=True)

## Boite à moustache

In [ ]:
def generer_boxplot(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]

    if df_filtered.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    grouped = df_filtered.copy()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.boxplot(x="Chaine", y="Temps_heures", data=grouped, palette="Set3")
    ax.set_title("Distribution du Temps d'émission (heures) par chaîne")
    ax.set_xlabel("Chaîne")
    ax.set_ylabel("Temps d'émission (heures)")
    plt.tight_layout()

    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    gr.Markdown("## Box Plot : Distribution du Temps d'Émission par Chaîne")

    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )

    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )

    btn = gr.Button("Générer le box plot")

    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_boxplot,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch(debug=True)

## Scatter plot

In [ ]:
def generer_scatterplot(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]

    if df_filtered.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    df_filtered["Temps_heures"] = df_filtered["Temps"] / 3600.0

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.scatterplot(x="Temps_heures", y="Theme", hue="Chaine", data=df_filtered, palette="Set1", s=100, ax=ax)
    ax.set_title("Relation entre le Theme et le Temps d'Émission par Chaîne")
    ax.set_xlabel("Nombre d'Émissions")
    ax.set_ylabel("Temps d'Émission (heures)")
    ax.legend(title="Chaîne")
    plt.tight_layout()

    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    gr.Markdown("## Scatter Plot : Nombre d'Émissions vs Temps d'Émission par Chaîne")

    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )

    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )

    btn = gr.Button("Générer le scatter plot")

    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_scatterplot,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch(debug=True)

## Tree map

In [ ]:
def generer_treemap(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]

    if df_filtered.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    # Agrégation par chaîne
    grouped = df_filtered.groupby("Chaine")["Temps"].sum().reset_index()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    # Création du treemap
    fig, ax = plt.subplots(figsize=(10, 6))
    squarify.plot(sizes=grouped["Temps_heures"], label=grouped["Chaine"], alpha=0.7, color=plt.cm.tab20.colors[:len(grouped)])
    plt.title("Treemap du Temps d'Émission (heures) par Chaîne")
    plt.axis('off')  # Désactive les axes pour une meilleure visualisation
    plt.tight_layout()

    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    gr.Markdown("## Treemap : Temps d'Émission par Chaîne")

    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )

    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )

    btn = gr.Button("Générer le treemap")

    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_treemap,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch(debug=True)

## Bubble chart

In [ ]:
def generer_bubble_chart(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]

    if df_filtered.empty:
        return "Aucune donnée après filtrage", df_filtered.head(10)

    fig = px.scatter(
        df_filtered,
        x="NbEmissions",
        y="Temps",
        size="Temps",
        color="Chaine",
        hover_name="Chaine",
        size_max=60,
        title="Bubble Chart : Nombre d'Émissions vs Temps d'Émission",
        labels={"NbEmissions": "Nombre d'Émissions", "Temps_heures": "Temps d'Émission (heures)"}
    )

    return fig, df_filtered.head(10)

with gr.Blocks() as demo:
    gr.Markdown("## Bubble Chart : Nombre d'Émissions vs Temps d'Émission par Chaîne")

    chaine_input = gr.CheckboxGroup(
        label="Sélectionnez la/les chaîne(s) :",
        choices=liste_chaines,
        value=liste_chaines
    )

    theme_input = gr.CheckboxGroup(
        label="Sélectionnez le/les thème(s) :",
        choices=liste_themes,
        value=liste_themes
    )

    btn = gr.Button("Générer le bubble chart")

    output_graph = gr.Plot()
    output_table = gr.DataFrame(label="Aperçu des données filtrées")

    btn.click(
        fn=generer_bubble_chart,
        inputs=[chaine_input, theme_input],
        outputs=[output_graph, output_table]
    )

demo.launch(debug=True)

# Intégrer plusieurs visuels dans un seul Dashboard

In [ ]:
def generer_histogramme(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]
    if df_filtered.empty:
        return "Aucune donnée après filtrage"

    grouped = df_filtered.groupby(["Annee", "Chaine"])["Temps"].sum().reset_index()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0
    df_pivot = grouped.pivot(index="Annee", columns="Chaine", values="Temps_heures").fillna(0)

    fig, ax = plt.subplots(figsize=(8, 6))
    df_pivot.plot(kind='bar', stacked=False, ax=ax, colormap='tab10')
    ax.set_title("Temps total d'émission (heures) par année et par chaîne")
    ax.set_xlabel("Année")
    ax.set_ylabel("Temps d'émission (heures)")
    ax.legend(title="Chaîne", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()

    return fig

def generer_camembert(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]
    if df_filtered.empty:
        return "Aucune donnée après filtrage"

    grouped = df_filtered.groupby("Chaine")["Temps"].sum().reset_index()
    grouped["Temps_heures"] = grouped["Temps"] / 3600.0

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(
        grouped["Temps_heures"],
        labels=grouped["Chaine"],
        autopct='%1.1f%%',
        startangle=140,
        colors=plt.colormaps['tab10'].colors[:len(grouped)]
    )
    ax.set_title("Répartition du temps d'émission (heures) par chaîne")
    ax.axis('equal')  # Assure que le camembert est circulaire
    plt.tight_layout()

    return fig

def generer_boxplot(chaine_selection, theme_selection):
    df_filtered = df[
        (df["Chaine"].isin(chaine_selection)) &
        (df["Theme"].isin(theme_selection))
    ]
    if df_filtered.empty:
        return "Aucune donnée après filtrage"

    df_filtered["Temps_heures"] = df_filtered["Temps"] / 3600.0

    fig, ax = plt.subplots(figsize=(8, 6))
    # Correction du FutureWarning en assignant 'hue' à 'Chaine' et en désactivant la légende
    sns.boxplot(x="Chaine", y="Temps_heures", data=df_filtered, hue="Chaine", palette="Set2", ax=ax)
    ax.set_title("Distribution du Temps d'Émission (heures) par Chaîne")
    ax.set_xlabel("Chaîne")
    ax.set_ylabel("Temps d'Émission (heures)")
    # Désactiver la légende car 'hue' est identique à 'x'
    #ax.legend_.remove()
    plt.tight_layout()

    return fig


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## Dashboard Multi-Visualisations : Analyse du Temps d'Émission")

    with gr.Row():
        with gr.Column():
            chaine_input = gr.CheckboxGroup(
                label="Sélectionnez la/les chaîne(s) :",
                choices=liste_chaines,
                value=liste_chaines  # sélection par défaut = toutes
            )
            theme_input = gr.CheckboxGroup(
                label="Sélectionnez le/les thème(s) :",
                choices=liste_themes,
                value=liste_themes  # sélection par défaut = tous
            )
            btn = gr.Button("Générer les graphiques")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Histogramme : Temps d'émission par Année et Chaîne")
            output_histogram = gr.Plot()
        with gr.Column():
            gr.Markdown("### Camembert : Répartition du Temps d'Émission par Chaîne")
            output_pie = gr.Plot()

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Box Plot : Distribution du Temps d'Émission par Chaîne")
            output_box = gr.Plot()
        with gr.Column():
            gr.Markdown("### Aperçu des Données Filtrées")
            output_table = gr.DataFrame(label="Aperçu des données filtrées")

    # Définir les actions lors du clic sur le bouton
    def generer_tous_les_graphiques(chaine_selection, theme_selection):
        fig_hist = generer_histogramme(chaine_selection, theme_selection)
        fig_pie = generer_camembert(chaine_selection, theme_selection)
        fig_box = generer_boxplot(chaine_selection, theme_selection)
        df_filtered = df[
            (df["Chaine"].isin(chaine_selection)) &
            (df["Theme"].isin(theme_selection))
        ].head(10)
        return fig_hist, fig_pie, fig_box, df_filtered

    btn.click(
        fn=generer_tous_les_graphiques,
        inputs=[chaine_input, theme_input],
        outputs=[output_histogram, output_pie, output_box, output_table]
    )

demo.launch(debug=True)